In [1]:
import pandas as pd
import numpy as np
import sys, os , time
from tabulate import tabulate

def find_match(string,list_des):
    return [value for value in list_des if string in value]

from IPython.display import display
def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(folder+file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
methods_list = ["Ann(tr)","softMV","hardMV", "DS", "Raykar", "OursGlobal", "OursIndividual","OursIndividual2","OursIndividual3"]
methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","Ours Individual","2", "3"]

def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(folder+archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return np.mean(result_list)
    except:
        return "-"
!ls

exp_Mchange_7groups.pickle  LabelMe_v2		run_CIFAR.sh
LabelMe			    LabelMe_v2_M2	run_LabelMe.sh
LabelMe_M2		    Plot_Results.ipynb	run_simple.sh


In [2]:
!ls LabelMe_v2
folder = "LabelMe_v2/"
files = sorted(os.listdir(folder))

LabelMe_DS_test.csv		      LabelMe_OursIndividual3_train.csv
LabelMe_DS_train.csv		      LabelMe_OursIndividual_testAux.csv
LabelMe_hardMV_test.csv		      LabelMe_OursIndividual_test.csv
LabelMe_hardMV_train.csv	      LabelMe_OursIndividual_trainAnn.csv
LabelMe_OursGlobal_testAux.csv	      LabelMe_OursIndividual_train.csv
LabelMe_OursGlobal_test.csv	      LabelMe_Raykar_test.csv
LabelMe_OursGlobal_trainAnn.csv       LabelMe_Raykar_trainAnn.csv
LabelMe_OursGlobal_train.csv	      LabelMe_Raykar_train.csv
LabelMe_OursIndividual2_testAux.csv   LabelMe_softMV_test.csv
LabelMe_OursIndividual2_test.csv      LabelMe_softMV_train.csv
LabelMe_OursIndividual2_trainAnn.csv  LabelMe_UpperBound_test.csv
LabelMe_OursIndividual2_train.csv     LabelMe_UpperBound_train.csv
LabelMe_OursIndividual3_testAux.csv   result_ind.txt
LabelMe_OursIndividual3_test.csv      result.txt
LabelMe_OursIndividual3_trainAnn.csv


In [7]:
for s in range(1,10):
    print("\n****************************** Scenario "+str(s)+" ******************************")
    scenario_files = find_match("s"+str(s)+".csv",files)
    for method in methods_list[1:-1]:
        aux = find_match(method,scenario_files)
        for value in aux:
            plot_files(value)
        print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="soft-MV"))




****************************** Scenario 1 ******************************
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334

****************************** Scenario 2 ******************************
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334

****************************** Scenario 3 ******************************
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334

****************************** Scenario 4 ******************************
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Epochs: 8.333333333333334
Ep

In [3]:
def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    for s in range(1,10):
        t["Scenario "+str(s)] = ["-" for _ in range(len(methods_list))]
        if tipo == 'acc':
            t.loc[0,"Scenario "+str(s)] =  calculate_aux("result_scenario"+str(s)+".txt",what_to_do="mv")

        scenario_files = find_match("s"+str(s)+".csv",files)

        for k, method in enumerate(methods_list[1:]):
            aux = find_match(method,scenario_files)
            if tipo == "acc": #como normal?
                test_aux = find_match("_test_",aux)
                if len(test_aux) != 0:
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+test_aux[0])["Accuracy"][0], 4)
                    
            elif tipo == 'norm' or tipo=='js':
                kl_aux = find_match("_train_",aux)
                if len(kl_aux) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    if tipo == 'norm':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"NormF"][0] ,4)
                        except:
                            pass
                    elif tipo=='js':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"JS"][0] ,4)
                        except:
                            pass
            elif tipo == 'accann':
                train_ann = find_match("_trainAnn_",aux)
                if len(train_ann) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_ann[0])[to_extract+"ACC imiting Annot"][0] ,4)
            elif tipo == 'rmse':
                train_aux = find_match("_train_",aux)
                if len(train_aux) != 0:
                    try:
                        t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_aux[0])["Average RMSE"][0] ,4)
                    except:
                        pass
            elif tipo == "epochs":
                t.loc[k+1,"Scenario "+str(s)]=calculate_aux("result_scenario"+str(s)+".txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_norm_mean = compare_table("norm","mean")
t_js_mean = compare_table("js","mean")
t_norm_wmean = compare_table("norm","wmean")
t_js_wmean = compare_table("js","wmean")
t_acc

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,OursIndividual,-,-,-,-,-,-,-,-,-
7,OursIndividual2,-,-,-,-,-,-,-,-,-
8,OursIndividual3,-,-,-,-,-,-,-,-,-


In [12]:
t_norm_wmean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,OursIndividual,-,-,-,-,-,-,-,-,-
7,OursIndividual2,-,-,-,-,-,-,-,-,-
8,OursIndividual3,-,-,-,-,-,-,-,-,-


In [12]:
t_js_mean #normal

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.2348,-,-,0.2307,0.212,-,0.2144,-,-
2,hardMV,0.1938,-,-,0.1984,0.2369,-,0.1448,-,-
3,DS,0.0044,-,-,0.0948,-,-,0.1087,-,-
4,Raykar,0.0042,-,-,0.0569,-,-,0.0475,-,-
5,OursGlobal,0.1966,-,-,0.2339,0.208,-,0.125,-,-
6,,0.0044,-,-,0.0948,0.208,-,0.1087,-,-


In [17]:
t_norm_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1474,-,-,0.145,0.1375,-,0.1398,-,-
2,hardMV,0.1376,-,-,0.1406,0.1566,-,0.1225,-,-
3,DS,0.014,-,-,0.0638,-,-,0.0674,-,-
4,Raykar,0.013,-,-,0.0583,-,-,0.0559,-,-
5,OursGlobal,0.1401,-,-,0.1548,0.1383,-,0.1109,-,-
6,,0.014,-,-,0.0638,0.1383,-,0.0674,-,-


In [20]:
t_accAnn_mean = compare_table("accann","mean")
t_accAnn_wmean = compare_table("accann","wmean")
t_rmseAnn = compare_table("rmse","normal")
t_accAnn_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,0.6856,-,-,0.7567,0.8267,-,0.7005,-,-
5,OursGlobal,0.6392,-,-,0.6807,0.741,-,0.6494,-,-
6,,0.6392,-,-,0.6807,0.741,-,0.6494,-,-


In [35]:
t_epochs = compare_table("epochs")
t_epochs

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [25]:
print(tabulate(t_acc,headers='keys',tablefmt='latex'))

\begin{tabular}{rlrrrrrrrll}
\hline
    & Methods    &   Scenario 1 &   Scenario 2 &   Scenario 3 &   Scenario 4 &   Scenario 5 &   Scenario 6 &   Scenario 7 & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    &     0.777371 &      0.61335 &      0.78022 &     0.771673 &     0.807082 &     0.622711 &     0.782255 & -            & -            \\
  1 & softMV     &     0.6644   &      0.6081  &      0.6572  &     0.6667   &     0.6764   &     0.5914   &     0.665    & -            & -            \\
  2 & hardMV     &     0.7248   &      0.5947  &      0.7527  &     0.7345   &     0.7297   &     0.5784   &     0.7913   & -            & -            \\
  3 & DS         &     0.9482   &      0.9429  &      0.9464  &     0.9344   &     0.9417   &     0.8413   &     0.9254   & -            & -            \\
  4 & Raykar     &     0.9415   &      0.9418  &      0.9417  &     0.9372   &     0.9308   &     0.8037   &     0.9179   & -            & -            \\
  5 & OursGlobal &     0.90

In [4]:
def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    t["Train"] = ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        t.loc[0,"Train"] =  calculate_aux("result.txt",what_to_do="mv")

    for k, method in enumerate(methods_list[1:]):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            test_aux = find_match("_train.csv",aux)
            t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
            train_aux = find_match("_test.csv",aux)
            t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            kl_aux = find_match("_train.csv",aux)
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            t.loc[k+1,"Train"]=calculate_aux("result.txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_norm_R = compare_table("NormF","R")
t_js_R = compare_table("JS","R")

t_norm_G = compare_table("NormF","G")
t_js_G = compare_table("JS","G")

t_epochs = compare_table("epochs")

t_acc

/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Methods,Train,Test
0,Ann(tr),0.769,-
1,softMV,0.8327,0.8191
2,hardMV,0.8042,0.8068
3,DS,0.1263,0.1408
4,Raykar,0.7802,0.7592
5,OursGlobal,0.8479,0.8285
6,OursIndividual,0.8051,0.8365
7,OursIndividual2,0.815,0.8348
8,OursIndividual3,0.7701,0.7643


In [5]:
from IPython.display import display
display(t_js_R)
display(t_norm_R)

,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,0.4632,-
2,hardMV,0.4924,-
3,DS,0.6273,-
4,Raykar,0.077,-
5,OursGlobal,0.4934,-
6,OursIndividual,0.5212,-
7,OursIndividual2,0.3592,-
8,OursIndividual3,0.3306,-


,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,0.2636,-
2,hardMV,0.2739,-
3,DS,0.3094,-
4,Raykar,0.1072,-
5,OursGlobal,0.2782,-
6,OursIndividual,0.2965,-
7,OursIndividual2,0.2319,-
8,OursIndividual3,0.2252,-


In [6]:
from IPython.display import display
display(t_js_G)
display(t_norm_G)

,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,0.0242,-
2,hardMV,0.0347,-
3,DS,0.4733,-
4,Raykar,0.4673,-
5,OursGlobal,0.0457,-
6,OursIndividual,0.0949,-
7,OursIndividual2,0.2102,-
8,OursIndividual3,0.1628,-


,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,0.0566,-
2,hardMV,0.0736,-
3,DS,0.2486,-
4,Raykar,0.2461,-
5,OursGlobal,0.0769,-
6,OursIndividual,0.1091,-
7,OursIndividual2,0.1695,-
8,OursIndividual3,0.136,-


In [35]:
t_epochs

,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,9.23333,-
2,hardMV,11.8667,-
3,DS,4.3,-
4,Raykar,3,-
5,OursGlobal,7.19,-
6,OursIndividual,NaN,-
7,OursIndividual2,8.42794,-
8,OursIndividual3,6.50521,-


In [5]:

for method,aux_meth in zip(methods_list[1:-1],methods_list_aux[:-1]):
    aux = find_match(method,files)
    for value in aux:
        plot_files(value)
    print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model=aux_meth))



File to show:  LabelMe_softMV_test.csv


,Unnamed: 0,Accuracy,F1 (micro)
0,All,0.816947,0.816947



File to show:  LabelMe_softMV_train.csv


,Unnamed: 0,Accuracy,F1 (micro),(R) NormF mean,(R) JS mean,(R) NormF w,(R) JS w,(G) NormF,(G) JS
0,All,0.8332,0.8332,0.172003,0.216288,0.121733,0.122006,0.056558,0.024225


Epochs: 8.33333333333

File to show:  LabelMe_hardMV_test.csv


...


File to show:  LabelMe_OursGlobal_train.csv


,Unnamed: 0,Accuracy,F1 (micro),(R) NormF mean,(R) JS mean,(R) NormF w,(R) JS w,(G) NormF,(G) JS,RMSE mean,RMSE w
0,All,0.8458,0.8458,0.188028,0.234098,0.137363,0.137175,0.085441,0.054124,0.149041,0.11153



File to show:  LabelMe_OursGlobal_trainAnn.csv


,ACC imiting Annot mean,Cross-entropy mean,ACC imiting Annot wmean,Cross entropy wmean
0,0.777385,0.613076,0.801034,0.569735


Epochs: 7.19833333333
